In [1]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

In [3]:

train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [4]:
print(train_data.shape)
print(test_data.shape) #28*28*1
train_features = torch.tensor(train_data.iloc[:, 1:][:].values, dtype=torch.float32)
test_features = torch.tensor(test_data[:].values, dtype=torch.float32)
train_labels = torch.tensor(
    train_data.label.values.reshape(-1, 1), dtype=torch.float32)
n_train = train_data.shape[0]

In [5]:
train_features = train_features.reshape(42000,1,28,28)
test_features = test_features.reshape(28000,1,28,28)
train_labels=train_labels.reshape(42000)

标签是1维，且不能为float。特征为（数量，通道数，宽，高）

In [6]:
# 划分数据集
from sklearn.model_selection import train_test_split
batch_size=256
X_train, X_test, y_train, y_test = train_test_split( train_features,train_labels, test_size=0.2)
train_iter = data.DataLoader(data.TensorDataset(*(X_train,y_train)), batch_size, shuffle=True)
test_iter = data.DataLoader(data.TensorDataset(*(X_test,y_test)), batch_size, shuffle=True)

# 定义模型

In [7]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))

In [8]:
# 评估函数
def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, nn.Module):
        net.eval()  # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [9]:
#@save
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型(在第六章定义)"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
#             print(epoch,i,y.dtype,y_hat.dtype)
            l = loss(y_hat, y.long())
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')

In [10]:
lr, num_epochs = 0.9, 10
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

# 预测

In [33]:
!pip install pandas==1.3.0

In [34]:
net.eval()
preds =net(test_features.to(d2l.try_gpu())).detach().cpu().numpy()
# 将其重新格式化以导出到Kaggle
import numpy as np
preds = np.argmax(preds,axis=1)
pre_size=preds.shape[0]
num = [i for i in range(1,pre_size+1)]
res_df=pd.DataFrame({
    'ImageId':num,
    'Label':preds
})
res_df.to_csv('submission.csv', index=False)